In [2]:
import numpy as np
import pandas
import torch
from joblib import load
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [3]:
data_new = pandas.read_pickle("./pickeddata.pkl")
data_new

,Date for US Imports and Exports,Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day),Weekly U.S. Exports of Crude Oil (Thousand Barrels per Day),Weekly U.S. Exports of Total Petroleum Products (Thousand Barrels per Day),Date for Retail Gas Price,Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),Season
0,9-Nov-01,10772,9639,1133,12-Nov-01,1.224,autumn
1,16-Nov-01,10243,8879,1364,19-Nov-01,1.208,autumn
2,23-Nov-01,9576,8187,1389,26-Nov-01,1.168,autumn
3,30-Nov-01,11170,9856,1314,3-Dec-01,1.149,autumn
4,7-Dec-01,9885,8966,919,10-Dec-01,1.136,winter
...,...,...,...,...,...,...,...
1137,25-Aug-23,-1684,2089,-3773,28-Aug-23,3.931,summer
1138,1-Sep-23,-2593,1838,-4432,4-Sep-23,3.925,autumn
1139,8-Sep-23,431,4492,-4061,11-Sep-23,3.941,autumn
1140,15-Sep-23,-2290,1450,-3741,18-Sep-23,4.001,autumn


In [4]:
season = {'spring': 0, 'summer': 1, 'autumn':2, 'winter': 3}
data_new.Season = [season[i] for i in data_new.Season]
data_new

,Date for US Imports and Exports,Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day),Weekly U.S. Exports of Crude Oil (Thousand Barrels per Day),Weekly U.S. Exports of Total Petroleum Products (Thousand Barrels per Day),Date for Retail Gas Price,Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),Season
0,9-Nov-01,10772,9639,1133,12-Nov-01,1.224,2
1,16-Nov-01,10243,8879,1364,19-Nov-01,1.208,2
2,23-Nov-01,9576,8187,1389,26-Nov-01,1.168,2
3,30-Nov-01,11170,9856,1314,3-Dec-01,1.149,2
4,7-Dec-01,9885,8966,919,10-Dec-01,1.136,3
...,...,...,...,...,...,...,...
1137,25-Aug-23,-1684,2089,-3773,28-Aug-23,3.931,1
1138,1-Sep-23,-2593,1838,-4432,4-Sep-23,3.925,2
1139,8-Sep-23,431,4492,-4061,11-Sep-23,3.941,2
1140,15-Sep-23,-2290,1450,-3741,18-Sep-23,4.001,2


In [5]:
XY=data_new[['Date for US Imports and Exports','Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day)','Season','Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)']].dropna(axis='index')
(feature1,featurename1)=('Date for US Imports and Exports',"Year")
(feature2,featurescale,featurename2)=('Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day)',1000,"Exports")
(feature3,featurename3)=('Season','Season')
(label,labelname)=('Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)',"Prices (Dollars per Gallon)")


XY.columns=[featurename1,featurename2,featurename3,labelname]
XY[featurename1]=pandas.to_datetime(XY[featurename1]).dt.year

print(XY)

X=XY[[featurename1,featurename2,featurename3]].squeeze()
Y=XY[labelname].squeeze()

      Year  Exports  Season  Prices (Dollars per Gallon)
0     2001    10772       2                        1.224
1     2001    10243       2                        1.208
2     2001     9576       2                        1.168
3     2001    11170       2                        1.149
4     2001     9885       3                        1.136
...    ...      ...     ...                          ...
1137  2023    -1684       1                        3.931
1138  2023    -2593       2                        3.925
1139  2023      431       2                        3.941
1140  2023    -2290       2                        4.001
1141  2023    -1706       2                        3.963

[1142 rows x 4 columns]


In [6]:
X = X.values
Y = Y.values

In [7]:
# Split the data
x_train, x_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [9]:
class pricePredictor_4layer(torch.nn.Module):
    def __init__(self,l1,l2,l3):
        super(pricePredictor_4layer, self).__init__()
        self.fc1 = torch.nn.Linear(3, l1)
        self.fc2 = torch.nn.Linear(l1, l2)
        self.fc3 = torch.nn.Linear(l2, l3)
        self.fc4 = torch.nn.Linear(l3, 1)
    def forward(self, x):
        x = self.fc1(x)
        x =torch.relu(x)
        x = self.fc2(x)
        x=torch.relu(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [10]:
cnn_model=pricePredictor_4layer(8,8,32)

In [12]:
cnn_model.load_state_dict(torch.load('cnn_dict'))
cnn_model.eval()

pricePredictor_4layer(
  (fc1): Linear(in_features=3, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)

**Feature Importance**

The permutation approach to feature importance evaluation may exhibit limitations, because the performance metrics calculated by permuting individual features do not provide meaningful insights into the actual importance of each feature. This is because the order of significance tends to fluctuate across iterations as also shown in the baseline model.

In [13]:
def shuffle_column(tensor, column_index):
    column_values = tensor[:, column_index]
    random_indices = torch.randperm(column_values.size(0))
    shuffled_column = column_values[random_indices]
    tensor[:, column_index] = shuffled_column
    return tensor

In [14]:
def compare_importance(model,x_tensor,y_tensor):
  loss_compare=np.zeros((x_tensor.shape[1]))
  n=1000
  loss_each_run=np.zeros((n))
  for i in range(x_tensor.shape[1]):
    for j in range(n):
      shuffled_tensor=shuffle_column(x_tensor, i)
      device = "cpu"
      if torch.cuda.is_available():
          device = "cuda:0"
      model.to(device)
      model.eval()
      output = model(x_tensor.to(device))
      metric = torch.nn.L1Loss()
      loss = metric(output, y_tensor.to(device))
      loss_each_run[j]=loss
    loss_compare[i]=np.average(loss_each_run)
  return loss_compare

In [15]:
loss_compare_val=compare_importance(cnn_model,x_val_tensor,y_val_tensor)

In [ ]:
print('CNN: Metric of each feature (after permutaion) ',loss_compare_val)
#[year, export, season]

CNN: Metric of each feature (after permutaion)  [0.56259565 0.62279789 0.63760864]


Feature Importance from Baseline Model

In [17]:
baseline = load('regression_deg9.joblib')

In [18]:
def compare_importance_baseline(base,deg,x_tensor,y_tensor):
    loss_compare=np.zeros((x_tensor.shape[1]))
    n=1000
    loss_each_run=np.zeros((n))
    for i in range(x_tensor.shape[1]):
      for j in range(n):
        shuffled_tensor=shuffle_column(x_tensor, i)
        input=shuffled_tensor.numpy()
        poly = PolynomialFeatures(degree=deg, include_bias=True)
        poly_features_val = poly.fit_transform(input)
        pred_Y = base.predict(poly_features_val)
        MAE=mean_absolute_error(y_tensor.numpy().reshape(-1), pred_Y)
        loss_each_run[j]=MAE
      loss_compare[i]=np.average(loss_each_run)
    return loss_compare

In [19]:
loss_baseline=compare_importance_baseline(baseline,9,x_val_tensor,y_val_tensor)

In [20]:
print('Baseline: Metric of each feature (after permutaion) ',loss_baseline)
#[year, export, season]

Baseline: Metric of each feature (after permutaion)  [1.6211844  1.64579947 1.70787445]
